In [1]:
from xarray import DataArray

from weldx import Q_, U_, GenericSeries
from pint import DimensionalityError

## Allowed variables (expressions only)

In [2]:
class AllowedVarsSeries(GenericSeries):
    _allowed_variables = ["a"]

In [3]:
try:
    AllowedVarsSeries("a*b")
except ValueError as e:
    print(e)

'['a', 'b']' is not a subset of the allowed expression variables ['a'] of class AllowedVarsSeries


In [4]:
AllowedVarsSeries("4*a");

## Required variables (expressions only)

In [5]:
class RequiredVarsSeries(GenericSeries):
    _required_variables = ["c"]

In [6]:
try:
    RequiredVarsSeries("a*b")
except ValueError as e:
    print(e)

RequiredVarsSeries requires expression variables '['c']'.


In [7]:
RequiredVarsSeries("a*c");

## Preprocess coordinates

In [8]:
def handle_time(instance, time, time_ref):
    from weldx import Time

    time = Time(time, time_ref)

    return dict(t=time.as_quantity())

In [9]:
class PreprocessCoordsSeries(GenericSeries):
    _evaluation_preprocessor = handle_time

### Discrete

In [10]:
pcs_discrete = PreprocessCoordsSeries(
    Q_([1, 2, 4], "m"), dims=["t"], coords={"t": Q_([2, 4, 7], "s")}
)

In [11]:
pcs_result_discrete = pcs_discrete(
    time="2000-01-01 15:00:05", time_ref="2000-01-01 15:00:00"
)
print(f"time: {pcs_result_discrete.data_array.t.data}")
print(f"data: {pcs_result_discrete.data}")

time: [5.]
data: [2.6666666666666665] m


In [12]:
try:
    pcs_discrete(x=3)
except TypeError:
    print("invalid input")

invalid input


### Expression

In [13]:
pcs_expr = PreprocessCoordsSeries("a * t", parameters=dict(a="3m/s"))

In [14]:
pcs_result_expr = pcs_expr(time="2000-01-01 15:00:05", time_ref="2000-01-01 15:00:00")
print(f"time: {pcs_result_expr.data_array.t.data}")
print(f"data: {pcs_result_expr.data}")

time: [5.]
data: [15.0] m


In [15]:
try:
    pcs_expr(x=3)
except TypeError:
    print("invalid input")

invalid input


## Required dimensions

In [16]:
class RequiredDimsSeries(GenericSeries):
    _required_dimensions = ["c"]

###  Discrete

In [17]:
try:
    RequiredDimsSeries(Q_([[1, 2], [3, 4]], "m"), ["a", "b"])
except ValueError as e:
    print(e)

RequiredDimsSeries requires dimensions '['c']'.


In [18]:
RequiredDimsSeries(Q_([[1, 2], [3, 4]], "m"), ["a", "c"]);

### Expression

In [19]:
try:
    RequiredDimsSeries("a*b")
except ValueError as e:
    print(e)

RequiredDimsSeries requires dimensions '['c']'.


In [20]:
RequiredDimsSeries("a*b", dims=dict(b="c"));

In [21]:
RequiredDimsSeries("a*c");

In [22]:
RequiredDimsSeries("a*b", parameters=dict(a=(Q_([1, 3], "m"), "c")));

## Required dimension units

In [23]:
class RequiredDimUnitsSeries(GenericSeries):
    _required_dimension_units = dict(t="s", c="m")

### Discrete

In [24]:
try:
    RequiredDimUnitsSeries(
        Q_([[1, 2], [3, 4]]),
        dims=["t", "c"],
    )
except KeyError as e:
    print(e)

"Could not find required coordinate 't'."


In [25]:
try:
    RequiredDimUnitsSeries(
        Q_([[1, 2], [3, 4]]),
        dims=["t", "c"],
        coords=dict(t=Q_([1, 2], "s"), c=Q_([1, 2], "K")),
    )
except DimensionalityError as e:
    print(e)

Cannot convert from 'K' to 'm'
Dimensionality mismatch in coordinate 'c'
Coordinate has unit 'K', expected 'm'


In [26]:
RequiredDimUnitsSeries(
    Q_([[1, 2], [3, 4]]),
    dims=["t", "c"],
    coords=dict(t=Q_([1, 2], "s"), c=Q_([1, 2], "m")),
);

### Expression

In [27]:
try:
    RequiredDimUnitsSeries("a*t + c")
except DimensionalityError as e:
    print(e)

Cannot convert from 'second' to 'meter'
Expression can not be evaluated due to a unit dimensionality error. Ensure that the expressions parameters and the expected variable units are compatible. The original exception was:
Cannot convert from 'second' ([time]) to 'meter' ([length])


In [28]:
try:
    RequiredDimUnitsSeries(
        "a*t + c", units=dict(t="K", c="m"), parameters=dict(a="m/K")
    )
except DimensionalityError as e:
    print(e)

Cannot convert from 's' to 'K'
RequiredDimUnitsSeries requires dimension t to have the unit dimensionality '[time]'


In [29]:
try:
    RequiredDimUnitsSeries(
        "a*t + c", units=dict(t="s"), parameters=dict(a="K/s", c="5K")
    )
except DimensionalityError as e:
    print(e)

Cannot convert from 'm' to 'K'
RequiredDimUnitsSeries requires dimension c to have the unit dimensionality '[length]'


In [30]:
RequiredDimUnitsSeries("a*t + c", units=dict(t="s", c="m"), parameters=dict(a="m/s"));

In [31]:
RequiredDimUnitsSeries("a*t + c", units=dict(t="s"), parameters=dict(a="m/s", c="m"));

In [32]:
RequiredDimUnitsSeries("a*t + c", units=dict(a="m/s"), parameters=dict(t="s", c="5m"));

## Required dimension coordinates

In [33]:
class RequiredDimCoordsSeries(GenericSeries):
    _required_dimension_coordinates = dict(c=["x", "y"])

### Discrete

In [34]:
try:
    RequiredDimCoordsSeries(
        Q_([[1, 2], [3, 4]]),
        dims=["t", "c"],
    )
except KeyError as e:
    print(e)

"Could not find required coordinate 'c'."


In [35]:
try:
    RequiredDimCoordsSeries(
        Q_([[1, 2], [3, 4]]), dims=["t", "c"], coords=dict(c=["a", "b"])
    )
except ValueError as e:
    print(e)

Value mismatch in DataArray and ref['c']


In [36]:
RequiredDimCoordsSeries(
    Q_([[1, 2], [3, 4]]), dims=["t", "c"], coords=dict(c=["x", "y"])
);

### Expression

In [37]:
try:
    RequiredDimCoordsSeries("a*t + c")
except ValueError as e:
    print(e)

RequiredDimCoordsSeries requires dimension c to have the coordinates ['x', 'y']. Therefore it can't be a variable dimension.


In [38]:
b = DataArray(Q_([1, 2], ""), dims=["c"])

try:
    RequiredDimCoordsSeries("a*t + b", parameters=dict(b=b))
except ValueError as e:
    print(e)

In [39]:
b = DataArray(Q_([1, 2], ""), dims=["c"], coords=dict(c=["a", "b"]))
try:
    RequiredDimCoordsSeries("a*t + b", parameters=dict(b=b))
except ValueError as e:
    print(e)

Value mismatch in DataArray and ref['c']


In [40]:
b = DataArray(Q_([1, 2], ""), dims=["c"], coords=dict(c=["x", "y"]))
RequiredDimCoordsSeries("a*t + b", parameters=dict(b=b));

## Required unit dimensionality

In [41]:
class RequiredUnitDimSeries(GenericSeries):
    _required_unit_dimensionality = U_("m")

### Discrete

In [42]:
try:
    RequiredUnitDimSeries(Q_("2K"))
except DimensionalityError as e:
    print(e)

Cannot convert from 'K' to 'm'
DataArray units are 'K'.  This is incompatible with the expected dimensionality '[length]'


In [43]:
RequiredUnitDimSeries(Q_("2m"));

### Expression

In [44]:
try:
    RequiredUnitDimSeries("a*b", units=dict(a="K"))
except DimensionalityError as e:
    print(e)

Cannot convert from 'K' to 'm'
RequiredUnitDimSeries requires its output unit to be of dimensionality '[length]' but it actually is '[temperature]'.


In [45]:
RequiredUnitDimSeries("a*b", units=dict(a="m"));